# Tutorial 4: Math Tool Agent

Large Language Models (LLMs) can serve as orchestrators, often referred to as
agents. In this tutorial, we will guide you step-by-step on how to blend
reasoning with the practical use of tools using GenC.

The `MathToolAgent` integrates reasoning with math tool, we will employ the
following components:

*   `ReAct` & `RepeatedConditionalChain` for facilitating the reasoning loop,
    enabling dynamic decision-making.
*   `RestCall`, accompanied by `parsers`, to illustrate making model calls and
    handling json request & response.
*   `LocalCache` is used to as a memory to recorde model interaction history.
*   `WolframAlpha`, utilized as a tool for answering mathematical queries.

This example is meant to demonstrate the essential ingredients to agents. For
production more robust prompt engineering is necessary.

## Define constants

In [ ]:
import textwrap
import generative_computing as genc

In [ ]:
gemini_api_key = ""
wolfram_app_id = ""


reasoning_template = """
  Solve a question answering task with interleaving Thought, Action, Observation steps.
  Thought can reason about the current situation
  Action can be only two types:
  (1) Math[query], Useful for when you need to solve math problems.
  (2) Finish[answer], which returns the answer as a number terminates.
  Here's an example:
  Question: what is the result of power of 2 + 1?
  Thought: power of 2
  Action: Math[pow(2)]
  Observation: 4
  Thought: I can now process the answer.
  Action: Math[4+1]
  Observation: 5
  Thought: Seems like I got the answer.
  Action: Finish[5]
  Please do it step by step.
  Question: {question}
  """

## Define the Agent

Agent behavior:

*   Template Instruct the model to break down a question into steps
*   Each step it explains its thought, and formulate a math equation as action
*   The equation is then calculated by Wolfram.
*   These interaction history are then stored in a memory/context
*   Repeat for another step...

In [ ]:
# Logger prints to terminal
log_it = genc.authoring.create_logger()

# Context keeps the interaction history with model.
# These Context is backed by a thread safe key value store.
read_context = genc.authoring.create_custom_function("/local_cache/read")
add_to_context = genc.authoring.create_custom_function("/local_cache/write")
evict_context = genc.authoring.create_custom_function("/local_cache/remove")

model_config = genc.authoring.create_rest_model_config(
    "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent",
    gemini_api_key,
)
model_call = genc.authoring.create_model_with_config(
    "/cloud/gemini", model_config
)

# Use WolframAlpha as a Tool to solve simple math questions.
solve_math = (
    genc.interop.langchain.CustomChain()
    | genc.authoring.create_custom_function("/react/extract_math_question")
    | genc.authoring.create_wolfram_alpha(wolfram_app_id)
    # Template Engine will extract the result from response JSON
    | genc.authoring.create_inja_template(
        "{% if queryresult.pods"
        " %}{{queryresult.pods.0.subpods.0.plaintext}}{% endif %}"
    )
)

# Define a resoning loop
# It will keep executing until a boolean op in the chain evaluates to true.
reasoning_loop = (
    genc.interop.langchain.CustomChain()
    | read_context
    | model_call
    | genc.authoring.create_custom_function("/react/parse_thought_action")
    | log_it
    | genc.authoring.create_regex_partial_match("Finish")
    | add_to_context
    | solve_math
    | genc.authoring.create_custom_function("/react/format_observation")
    | log_it
    | add_to_context
)
# If agent can't get an answer by 8th iteration, terminate.
reasoning_loop.num_iteration = 8

# Now set up the instruction tempalte and wire the agent together.
instruction_template = genc.authoring.create_prompt_template(
    reasoning_template
)

math_agent_chain = (
    genc.interop.langchain.CustomChain()
    | evict_context
    | instruction_template
    | add_to_context
    | log_it
    | reasoning_loop
)

# Compiles the agent into a portable computation, which can be run in C++/mobile
portable_ir = genc.interop.langchain.create_computation(math_agent_chain)

## Compile & Run it

In [ ]:
# Runs the agent
runner = genc.runtime.Runner(portable_ir,
                             genc.examples.executor.create_default_executor())
runner(
    """what is the result of (square root of 4) + 3 to the power of 2 + 3 *(8 +
     4) / 2 - 7"""
)

**Sample Output**

*   Thought: I need to calculate the square root of 4 and cube 3 first.
*   Action: Math[sqrt(4)]
*   Observation: 2
*   Thought: I need to cube 3 now.
*   Action: Math[pow(3, 2)]
*   Observation: 9
*   Thought: Now I need to evaluate the expression.
*   Action: Math[(2) + (9) + (3 * (8 + 4) / 2) - 7]
*   Observation: 22
*   Thought: Seems like I got the answer.
*   Action: Finish[22]

## Save the IR to a file and deploy on Android

1. Run following code to save the IR to a file.

2. Follow instructions [here](.../tutorial_1_simple_cascadeipynb#scrollTo=GhLV1BlXt0fK) to deploy the IR on Android phone.

In [ ]:
from google3.pyglib import gfile

with gfile.Open("/tmp/genc_demo.pb", "wb") as f:
  f.write(portable_ir.SerializeToString())

## Run the demo on Android

Open the “Generative Computing Demo” app and type a math query.
Sample query to enter in UI:

```What is the result of (square root of 4) + 3 to the power of 2```

See the result.

## Cross Language Interoperability

*   The portable computation, can be run in C++ and Java
*   Similar Chaining authoring interface is avaiable in C++, if you wish to
    replicate this logic in C++, you can find a full example under cc/example

## Intro to the next tutorial: Combine C++ Runtime with LangChain Interface

In this tutorial we used GenC authoring interfaces, which transparently shows
the exact logic under an agent.

Developer may prefer other authoring framework such as LangChain. And you may
have also noticed that the tools vary from use case to use case, but reasoning
loop maybe reusable. How can we levarage the benefit of LangChain?

In the next tutorial we'll demonstrate how the above code could be combined with
LangChain interface to achieve both brevity of LangChain, robustness and
transparency of GenC.